# 4.10 Coding Etiquette & Excel Reporting

## This script contains the following points:

      1. Part 1.1 Import datasets
      2. Part 1.2 Address PII data
      3. Part 1.3 Create regional segmentation
      4. Part 1.4 Create exclusion flag for low activity customers
      5. Part 1.5 Create customer profiles based on age, income, number of dependents, and family status

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# create path shortcut for data import
path = r'/Users/sydneyhoy/Documents/Data Immersion/Achievement 4/02-2024 Instacart Basket Analysis'

## Part 1.1 Import datasets

In [3]:
# import dataset customers_merged dataset
customers_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customers_merged.pkl'))

In [4]:
# check shape of dataset
customers_merged.shape

(30328763, 32)

In [5]:
# check output
customers_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income,_merge
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [6]:
# check data types
customers_merged.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
price_range_loc             object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
avg_order                  float64
spending_flag               object
median_order_frequency     float64
order_frequency             object
First Name                  object
Last Name                   object
Gender                      object
State                       object
Age                          int64
date_joined                 object
n_dependants        

## Part 1.2 Address PII data

In [7]:
# drop unnecessary colomns and PII data First Name and Last Name
customers_merged = customers_merged.drop(columns = ['_merge','First Name','Last Name'])

## Part 1.3 Create regional segmentation

In [8]:
# set northeast region flag
customers_merged.loc[customers_merged['State'].isin(['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']),'Region'] = 'Northeast'

In [9]:
# set midwest region flag
customers_merged.loc[customers_merged['State'].isin(['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota','Iowa', 'Missouri']),'Region'] = 'Midwest'

In [10]:
# set south region flag
customers_merged.loc[customers_merged['State'].isin(['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee','Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']),'Region'] = 'South'

In [11]:
# set west region flag
customers_merged.loc[customers_merged['State'].isin(['Idaho', 'Montana', 'Wyoming','Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska','Washington', 'Oregon', 'California', 'Hawaii']),'Region'] = 'West'

In [12]:
# check value counts
customers_merged['Region'].value_counts(dropna = False)

Region
South        10099817
West          7765088
Midwest       7107586
Northeast     5356272
Name: count, dtype: int64

In [13]:
# create crosstab for region and spending flag
crosstab = pd.crosstab(customers_merged['Region'], customers_merged['spending_flag'], dropna = False)

In [14]:
# export crosstab
crosstab.to_clipboard()

## Part 1.4 Create exclusion flag for low activity customers

In [15]:
# create a low activity flag for customers with less than 5 orders
customers_merged.loc[customers_merged['max_order'] < 5, 'customer activity'] = 'Low Activity'

In [16]:
# create a normal activity flag for customers with 5 orders or more
customers_merged.loc[customers_merged['max_order'] >= 5, 'customer activity'] = 'Normal Activity'

In [17]:
# check output
customers_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency,Gender,State,Age,date_joined,n_dependants,fam_status,income,Region,customer activity
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity


In [18]:
# create subset excluding low activity customers
customers_merged_activity = customers_merged[customers_merged['customer activity'] == 'Normal Activity']

In [19]:
# export data to pkl
customers_merged_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customers_merged_high_activity.pkl'))

In [20]:
# check shape
customers_merged_activity.shape

(29303281, 31)

In [21]:
# check output
customers_merged_activity.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency,Gender,State,Age,date_joined,n_dependants,fam_status,income,Region,customer activity
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Normal Activity


## Part 1.5 Create customer profiles based on age, income, number of dependents, and family status

In [22]:
# aggregate age get information to create profiles
customers_merged_activity.agg({'Age': ['mean', 'min', 'max']})

,Age
mean,49.465938
min,18.000000
max,81.000000


In [23]:
# aggregate income get infrmation to create profiles
customers_merged_activity.agg({'income': ['mean', 'min', 'max']})

,income
mean,99720.592337
min,25903.000000
max,593901.000000


In [24]:
# aggregate n_dependants get information to create profiles
customers_merged_activity.agg({'n_dependants': ['mean', 'min', 'max']})

,n_dependants
mean,1.5019
min,0.0000
max,3.0000


In [25]:
# aggregate fam_status get information to create profiles
customers_merged_activity['fam_status'].value_counts(dropna = False)

fam_status
married                             20575742
single                               4820773
divorced/widowed                     2504114
living with parents and siblings     1402652
Name: count, dtype: int64

In [26]:
# check aggregates of n_dependants grouped by fam_status to create profiles
customers_merged_activity.groupby('fam_status').agg({'n_dependants': ['mean', 'min', 'max']})

n_dependants        
                                         mean min max
fam_status                                           
divorced/widowed                     0.000000   0   0
living with parents and siblings     1.986528   1   3
married                              2.003534   1   3
single                               0.000000   0   0

In [27]:
# check aggregates of age grouped by fam_status to create profiles
customers_merged_activity.groupby('fam_status').agg({'Age': ['mean', 'min', 'max']})

Age        
                                       mean min max
fam_status                                         
divorced/widowed                  70.511518  60  81
living with parents and siblings  19.502122  18  21
married                           51.491379  22  81
single                            38.607376  18  59

In [28]:
# create a dependants flag 
customers_merged_activity.loc[customers_merged_activity['n_dependants'] > 0, 'Dependants'] = 'Yes'

/var/folders/wd/xh8m0hc916l1ffmsssspd3f40000gn/T/ipykernel_86735/2654910760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_merged_activity.loc[customers_merged_activity['n_dependants'] > 0, 'Dependants'] = 'Yes'


In [29]:
customers_merged_activity.loc[customers_merged_activity['n_dependants'] == 0, 'Dependants'] = 'No'

In [30]:
# check value counts
customers_merged_activity['Dependants'].value_counts(dropna = False)

Dependants
Yes    21978394
No      7324887
Name: count, dtype: int64

In [44]:
# create an array for income to find the 25th 50th and 75th percentile to create income flag
array = customers_merged_activity['income']

In [45]:
array

0           40423
1           40423
2           40423
3           40423
4           40423
            ...  
30328755    53755
30328756    53755
30328757    53755
30328758    53755
30328759    53755
Name: income, Length: 29303281, dtype: int64

In [46]:
# 25th percentile
np.percentile(array, 25)

67351.0

In [47]:
# 50th percentile
np.percentile(array,50)

96780.0

In [48]:
# 75th percentile
np.percentile(array,75)

128115.0

In [49]:
# create an income range of low, mid, and high income based on percentiles
customers_merged_activity.loc[customers_merged_activity['income'] <= 67351, 'Income Range'] = 'Low Income'

/var/folders/wd/xh8m0hc916l1ffmsssspd3f40000gn/T/ipykernel_86735/3562284314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_merged_activity.loc[customers_merged_activity['income'] <= 67351, 'Income Range'] = 'Low Income'


In [50]:
customers_merged_activity.loc[(customers_merged_activity['income'] > 67351) & (customers_merged_activity['income'] < 128115), 'Income Range'] = 'Mid Income'

In [51]:
customers_merged_activity.loc[customers_merged_activity['income'] >= 128115, 'Income Range'] = 'High Income'

In [52]:
# check value counts
customers_merged_activity['Income Range'].value_counts(dropna = False)

Income Range
Mid Income     14651459
High Income     7325998
Low Income      7325824
Name: count, dtype: int64

In [53]:
# create income profile based on income , fam_status, and dependents
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Low Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'Low income single with dependants'

/var/folders/wd/xh8m0hc916l1ffmsssspd3f40000gn/T/ipykernel_86735/1472014248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Low Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'Low income single with dependants'


In [54]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Mid Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'Mid income single with dependants'

In [55]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'High Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'High income single with dependants'

In [56]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Low Income') & (customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'Low income single without dependants'

In [57]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Mid Income') & (customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'Mid income single without dependants'

In [58]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'High Income') & (customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Income Profile'] = 'High income single without dependants'

In [59]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Low Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'] == 'married'), 'Income Profile'] = 'Low income married with dependants'

In [60]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Mid Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'] == 'married'), 'Income Profile'] = 'Mid income married with dependants'

In [61]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'High Income') & (customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'] == 'married'), 'Income Profile'] = 'High income married with dependants'

In [62]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Low Income') & (customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'] == 'married'), 'Income Profile'] = 'Low income married without dependants'

In [63]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'Mid Income') & (customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'] == 'married'), 'Income Profile'] = 'Mid income married without dependants'

In [64]:
customers_merged_activity.loc[(customers_merged_activity['Income Range'] == 'High Income') & (customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'] == 'married'), 'Income Profile'] = 'High income married without dependants'

In [65]:
# check value counts
customers_merged_activity['Income Profile'].value_counts(dropna = False)

Income Profile
Mid income married with dependants       10238804
High income married with dependants       5467272
Low income married with dependants        4869666
Mid income single without dependants      3657168
High income single without dependants     1841579
Low income single without dependants      1826140
Mid income single with dependants          755487
Low income single with dependants          630018
High income single with dependants          17147
Name: count, dtype: int64

In [79]:
# create family profile based on fam_status and dependants
customers_merged_activity.loc[(customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Family Profile'] = 'Single with dependants'

In [80]:
customers_merged_activity.loc[(customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Family Profile'] = 'Single without dependants'

In [81]:
customers_merged_activity.loc[(customers_merged_activity['Dependants'] == 'Yes') & (customers_merged_activity['fam_status'] =='married'), 'Family Profile'] = 'Married with dependants'

In [82]:
customers_merged_activity.loc[(customers_merged_activity['Dependants'] == 'No') & (customers_merged_activity['fam_status'] =='married'), 'Family Profile'] = 'Married without dependants'

In [83]:
# Create age profile based on fam_status and age
customers_merged_activity.loc[(customers_merged_activity['Age'] <= 35) & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Age Profile'] = 'Single adult under 36'

In [84]:
customers_merged_activity.loc[(customers_merged_activity['Age'] > 35) & (customers_merged_activity['Age'] < 60) & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Age Profile'] = 'Single adult 36-59'

In [85]:
customers_merged_activity.loc[(customers_merged_activity['Age'] >= 60) & (customers_merged_activity['fam_status'].isin (['single', 'living with parents and siblings', 'divorced/widowed'])), 'Age Profile'] = 'Single adult 60+'

In [86]:
customers_merged_activity.loc[(customers_merged_activity['Age'] <= 35) & (customers_merged_activity['fam_status'] =='married'), 'Age Profile'] = 'Married adult under 36'

In [87]:
customers_merged_activity.loc[(customers_merged_activity['Age'] > 35) & (customers_merged_activity['Age'] < 60) & (customers_merged_activity['fam_status'] =='married'), 'Age Profile'] = 'Married adult 36-59'

In [88]:
customers_merged_activity.loc[(customers_merged_activity['Age'] >= 60) & (customers_merged_activity['fam_status'] =='married'), 'Age Profile'] = 'Married adult over 60'

In [89]:
# check age profile value counts
customers_merged_activity['Age Profile'].value_counts(dropna = False)

Age Profile
Married adult 36-59       8266877
Married adult over 60     7500912
Married adult under 36    4807953
Single adult under 36     3462704
Single adult 36-59        2760721
Single adult 60+          2504114
Name: count, dtype: int64

In [90]:
#check family profile value counts
customers_merged_activity['Family Profile'].value_counts(dropna = False)

Family Profile
Married with dependants      20575742
Single without dependants     7324887
Single with dependants        1402652
Name: count, dtype: int64

In [91]:
# export data as instacart final to pickle
customers_merged_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'instacart_profiles.pkl'))